<a href="https://colab.research.google.com/github/OptimizationExpert/Pyomo/blob/main/MagicSquare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Developed by Alireza Soroudi
alireza.soroudi@gmail.com

Linkedin News letter : https://www.linkedin.com/newsletters/optimization-in-open-source-6874020019009859585/

Git hub: https://github.com/OptimizationExpert/Pyomo



In [1]:
!pip install ortools
from ortools.sat.python import cp_model # CP-SAT solver
import numpy as np
from random import randint
import random
from math import ceil
import pandas as pd
import matplotlib.pyplot as plt # Data visualization
from matplotlib.patches import Rectangle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 23.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.2 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
google-api-core 2.11.1 requires protobuf!=3.20.0,!=3.20.1,!=4.2

A magic square is a fascinating mathematical puzzle where numbers are arranged in a grid in such a way that the sum of the numbers in each row, column, and diagonal is the same. It's like a numerical crossword, where every row and column holds a unique combination of numbers, and the challenge lies in arranging the digits so that they magically add up to a constant sum. Magic squares come in various sizes and complexities, from simple 3x3 grids to larger, intricate puzzles. Solving a magic square requires both logic and creativity, making it a delightful and engaging mathematical challenge for enthusiasts of all ages. M = n(n*n+1)/2

In [2]:
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""
    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0
    def on_solution_callback(self):
        self.__solution_count += 1
        N = int(np.sqrt(len(self.__variables)))
        for r in range(1,1+N):
            print([self.Value(self.__variables[i,j]) for (i,j) in self.__variables if i==r])
        print('-------------------------')
    def solution_count(self):
        return self.__solution_count

def SearchForAllSolutionsSampleSat(n):
    """Showcases calling the solver to search for all solutions."""
    # Creates the model.
    model = cp_model.CpModel()

    # Creates the variables.
    M = int(n*(n*n+1)/2)
    rows = range(1,n+1)
    cols = range(1,n+1)
    x = {(i,j):model.NewIntVar(1, n**2, f"x_{i}_{j}") for i in rows for j in cols}

    # Create the constraints.
    model.AddAllDifferent([x[i,j] for (i,j) in x])

    for r in rows:
        expressions = [x[r,c] for c in cols]
        model.Add(sum(expressions) == M )
        model.AddAllDifferent(expressions)


    for c in cols:
        expressions = [x[r,c] for r in rows]
        model.Add(sum(expressions) == M )
        model.AddAllDifferent(expressions)

    expressions_d = [x[r,c] for r in rows for c in cols if r==c]
    model.Add(sum(expressions_d) == M )

    expressions_ad = [x[r,c] for r in rows for c in cols if r==n-c+1]
    model.Add(sum(expressions_ad) == M )


    # Create a solver and solve.
    solver = cp_model.CpSolver()
    solution_printer = VarArraySolutionPrinter(x)
    # Enumerate all solutions.
    solver.parameters.enumerate_all_solutions = True
    # Solve.

    status = solver.Solve(model, solution_printer)
    #status = solver.Solve(model)

    print(f"Status = {solver.StatusName(status)}")
    print(f"Number of solutions found: {solution_printer.solution_count()}")


SearchForAllSolutionsSampleSat(3)

[2, 9, 4]
[7, 5, 3]
[6, 1, 8]
-------------------------
[4, 9, 2]
[3, 5, 7]
[8, 1, 6]
-------------------------
[6, 7, 2]
[1, 5, 9]
[8, 3, 4]
-------------------------
[2, 7, 6]
[9, 5, 1]
[4, 3, 8]
-------------------------
[4, 3, 8]
[9, 5, 1]
[2, 7, 6]
-------------------------
[8, 3, 4]
[1, 5, 9]
[6, 7, 2]
-------------------------
[8, 1, 6]
[3, 5, 7]
[4, 9, 2]
-------------------------
[6, 1, 8]
[7, 5, 3]
[2, 9, 4]
-------------------------
Status = OPTIMAL
Number of solutions found: 8
